<a href="https://colab.research.google.com/github/purbid/NLP-projects/blob/main/entailment_example_with_research_paper_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.2 MB/s 
     |████████████████████████████████| 101 kB 12.2 MB/s 
     |████████████████████████████████| 6.6 MB 41.0 MB/s 
     |████████████████████████████████| 596 kB 63.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import string
import random
from collections import defaultdict
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
encoder = AutoModel.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/entialment_model_example/entailment_multi_class_ds.csv')

In [7]:
df.columns

Index(['Title', 'Conference'], dtype='object')

In [6]:
descriptions_dict = {'VLDB':'This is storage databases VLDB',
  'ISCAS' : 'This is circuits and systems ISCAS',
  'SIGGRAPH':'This is graohics and interactive design SIGGRAPH',
  'INFOCOM' : 'This is networks INFOCOM',
  'WWW' : 'This is query and graph WWW'}

In [15]:
all_x, all_y, entailment_texts = [], [], []

for itr, row in df.iterrows():
 

  all_x.append(row['Title'])
  all_x.append(row['Title'])

  entailment_texts.append(descriptions_dict[row['Conference']])
  all_y.append(1)

  candidate_list = []
  for desc in descriptions_dict:
    if desc.lower()!= row['Conference'].lower():
      candidate_list.append(desc)
  negative_label = random.choice(candidate_list)
  entailment_texts.append(descriptions_dict[negative_label])
  
  all_y.append(0)

In [16]:
print(len(all_x), len(all_y), len(entailment_texts))

5014 5014 5014


In [17]:
train_data_size = int(0.8*len(all_x))
print('training data size is  {} '.format(train_data_size))

training data size is  4011 


In [18]:
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

In [19]:
train_encodings = tokenizer(all_x[:train_data_size], entailment_texts[:train_data_size], max_length = 400, pad_to_max_length=True, truncation=True)
val_encodings = tokenizer(all_x[train_data_size:], entailment_texts[train_data_size:], max_length = 400, pad_to_max_length=True, truncation=True)


class RhetDataSet(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = RhetDataSet(train_encodings, all_y[:train_data_size])
val_dataset = RhetDataSet(val_encodings, all_y[train_data_size:])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [29]:
###### let's see how the training data looks like ###
for i in range(6):
  print(str(tokenizer.convert_ids_to_tokens(train_dataset[i]['input_ids'])).split('[PAD]')[0] + '\t' + str(train_dataset[i]['labels']))

['[CLS]', 'innovation', 'in', 'database', 'management', ':', 'computer', 'science', 'vs', '.', 'engineering', '.', '[SEP]', 'this', 'is', 'storage', 'databases', 'v', '##ld', '##b', '[SEP]', '	tensor(1)
['[CLS]', 'innovation', 'in', 'database', 'management', ':', 'computer', 'science', 'vs', '.', 'engineering', '.', '[SEP]', 'this', 'is', 'query', 'and', 'graph', 'www', '[SEP]', '	tensor(0)
['[CLS]', 'high', 'performance', 'prime', 'field', 'multiplication', 'for', 'gp', '##u', '.', '[SEP]', 'this', 'is', 'circuits', 'and', 'systems', 'is', '##cas', '[SEP]', '	tensor(1)
['[CLS]', 'high', 'performance', 'prime', 'field', 'multiplication', 'for', 'gp', '##u', '.', '[SEP]', 'this', 'is', 'networks', 'info', '##com', '[SEP]', '	tensor(0)
['[CLS]', 'enchanted', 'scissors', ':', 'a', 'sci', '##sso', '##r', 'interface', 'for', 'support', 'in', 'cutting', 'and', 'interactive', 'fabrication', '.', '[SEP]', 'this', 'is', 'gr', '##ao', '##hic', '##s', 'and', 'interactive', 'design', 'si', '##gg',

In [41]:
device = 'cuda'
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels = 2).to(device)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.21.1",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9c169103d7e5a

In [42]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/IIT_law_ai/semantic_segmentation/sentence_level_entailment/trained_models/',          # output directory
    num_train_epochs=7,              # total number of training epochs
    evaluation_strategy= "epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='/content/drive/MyDrive/IIT_law_ai/semantic_segmentation/sentence_level_entailment/trained_models/logs/',            # directory for storing logs
    logging_steps=10,
    load_best_model_at_end=True
)
trainer = Trainer(
    model=model.to(device),                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [43]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4011
  Num Epochs = 7
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3514


Epoch,Training Loss,Validation Loss
1,0.405500,0.411433
2,0.255000,0.396311
3,0.226000,0.416536
4,0.235800,0.565296
5,0.072200,0.592807
6,0.086800,0.600640
7,0.146700,0.634720


***** Running Evaluation *****
  Num examples = 1003
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/IIT_law_ai/semantic_segmentation/sentence_level_entailment/trained_models/checkpoint-502
Configuration saved in /content/drive/MyDrive/IIT_law_ai/semantic_segmentation/sentence_level_entailment/trained_models/checkpoint-502/config.json
Model weights saved in /content/drive/MyDrive/IIT_law_ai/semantic_segmentation/sentence_level_entailment/trained_models/checkpoint-502/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1003
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/IIT_law_ai/semantic_segmentation/sentence_level_entailment/trained_models/checkpoint-1004
Configuration saved in /content/drive/MyDrive/IIT_law_ai/semantic_segmentation/sentence_level_entailment/trained_models/checkpoint-1004/config.json
Model weights saved in /content/drive/MyDrive/IIT_law_ai/semantic_segmentation/sentence_level_entailment/trained_models/checkpoint-10

TrainOutput(global_step=3514, training_loss=0.2479543850800117, metrics={'train_runtime': 715.2351, 'train_samples_per_second': 39.256, 'train_steps_per_second': 4.913, 'total_flos': 2905693087550400.0, 'train_loss': 0.2479543850800117, 'epoch': 7.0})

In [44]:
from sklearn import metrics
import numpy as np
predictions_full, labels, _ = trainer.predict(val_dataset)
predictions = np.argmax(predictions_full, axis = 1)
print(metrics.classification_report(labels, predictions))

***** Running Prediction *****
  Num examples = 1003
  Batch size = 8


              precision    recall  f1-score   support

           0       0.84      0.90      0.87       502
           1       0.89      0.82      0.86       501

    accuracy                           0.86      1003
   macro avg       0.86      0.86      0.86      1003
weighted avg       0.86      0.86      0.86      1003



In [45]:
from sklearn import metrics

def add_negative_examples_for_validation(input_sentence, label, entailment_text):
    curr_input_all, curr_labels_all, curr_entailment_texts_all = [],  [], [] #1, 1, 1
    real_label = entailment_text.split(' ')[-1]
    for description in descriptions_dict :
      if description.lower()!=real_label.lower():
        curr_entailment_texts_all.append(descriptions_dict[description])
        curr_labels_all.append(0)
        curr_input_all.append(input_sentence)

    random_index_for_true_text = random.choice(range(len(descriptions_dict)-1))
    # print(random_index_for_true_text)
    curr_input_all.insert(random_index_for_true_text, input_sentence)
    curr_labels_all.insert(random_index_for_true_text, 1)
    curr_entailment_texts_all.insert(random_index_for_true_text, entailment_text)

    return curr_input_all, curr_labels_all, curr_entailment_texts_all


def pick_sentence_with_highest_probability(probs_tensor):


  probabilities_sub_tensor = probs_tensor #prospective_sentences*n_tags
  pos = probabilities_sub_tensor.max(dim=1).values.argmax()
  # print(probs_tensor)
  preds_sublist = probs_tensor.argmax(dim = 1)
  # print(preds_sublist)


  idx_list = []
  
  for pred in preds_sublist:
    if pred == 0:
      idx_list.append([False]*2)
    else:
      idx_list.append([True]*2)
  condition  = torch.tensor(idx_list)
  
  probabilities_sub_tensor = probabilities_sub_tensor.where(condition.to('cuda'), torch.tensor(-1000.0).to('cuda'))  
  probabilities_sub_tensor = probabilities_sub_tensor.max(dim=1)
  pos = probabilities_sub_tensor.values.argmax() 
  
  return pos

def validate_examples(list_of_sentences = [], list_of_labels = [], list_of_entailment_texts = []):
 
  real_labels, pred_labels = [], []
  print('validating on model, output will be label wise F1') #validation
  
  assert len(list_of_sentences) == len(list_of_labels), "Length of labels doesn't match inputs" 

  model.eval()
  print('total sentences are : {}'.format(len(list_of_sentences)//2))
  sent_num = 0
  for sent, label, entailment_text in zip(list_of_sentences, list_of_labels, list_of_entailment_texts):
    if sent_num%100==0:
      print(sent_num)
    sent_num+=1
    if label == 0:
      continue # we don't want negative labels as of now
    else:
      curr_sentence_all_set, all_labels_set, curr_entailment_texts_all = add_negative_examples_for_validation(sent, label, entailment_text)
      # print(curr_sentence_all_set)
      # print('\n')
      # print(curr_entailment_texts_all)
      validation_input = tokenizer(curr_sentence_all_set, curr_entailment_texts_all, max_length = 400, pad_to_max_length=True, truncation=True, return_tensors='pt').to(device)
      with torch.no_grad():
        class_scores = model(**validation_input)
      pred_label_pos = pick_sentence_with_highest_probability(class_scores.logits)
      real_labels.append(entailment_text.split(' ')[-1])
      pred_labels.append(curr_entailment_texts_all[pred_label_pos].split(" ")[-1])
  
  print(metrics.classification_report(real_labels, pred_labels))

probs = validate_examples(all_x[train_data_size:], all_y[train_data_size:], entailment_texts[train_data_size:])   


validating on model, output will be label wise F1
total sentences are : 501
0


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


100
200
300
400
500
600
700
800
900
1000
              precision    recall  f1-score   support

     INFOCOM       0.86      0.75      0.81       110
       ISCAS       0.75      0.89      0.82       146
    SIGGRAPH       0.76      0.66      0.71        68
        VLDB       0.59      0.72      0.65        83
         WWW       0.82      0.62      0.70        94

    accuracy                           0.75       501
   macro avg       0.76      0.73      0.74       501
weighted avg       0.76      0.75      0.75       501

